In [2]:
import pandas as pd

In [3]:
df = pd.read_csv('btcusd_1-min_data.csv')

In [4]:
df

,Timestamp,Open,High,Low,Close,Volume
0,1.325412e+09,4.58,4.58,4.58,4.58,0.000000
1,1.325412e+09,4.58,4.58,4.58,4.58,0.000000
2,1.325412e+09,4.58,4.58,4.58,4.58,0.000000
3,1.325412e+09,4.58,4.58,4.58,4.58,0.000000
4,1.325412e+09,4.58,4.58,4.58,4.58,0.000000
...,...,...,...,...,...,...
7057163,1.748911e+09,106290.00,106290.00,106256.00,106278.00,0.023115
7057164,1.748912e+09,106277.00,106349.00,106277.00,106349.00,0.803444
7057165,1.748912e+09,106328.00,106384.00,106328.00,106384.00,0.007128
7057166,1.748912e+09,106358.00,106371.00,106352.00,106363.00,0.399529


In [5]:
df.isnull().sum()

Timestamp    0
Open         0
High         0
Low          0
Close        0
Volume       0
dtype: int64

In [6]:
df["Timestamp"] = pd.to_datetime(df["Timestamp"], unit='s')
df.set_index("Timestamp", inplace = True)

df_day = df.resample("D").mean()

In [7]:
df_day.drop(columns=['Open', 'High', 'Low', 'Volume'], inplace=True)

In [8]:
df_day

,Close
Timestamp,
2012-01-01,4.645697
2012-01-02,4.975000
2012-01-03,5.085500
2012-01-04,5.170396
2012-01-05,5.954361
...,...
2025-05-30,105314.915972
2025-05-31,104123.672917
2025-06-01,104646.463194


In [9]:
import plotly.express as px
fig = px.line(x=df_day.index, y=df_day['Close'], labels={'x': 'Ngày', 'y': 'Giá đóng cửa'}, title='Chuỗi giá đóng cửa Bitcoin hàng ngày')
fig.show()

In [14]:
cutoff = int(len(df_day) * 0.8)
df_train = df_day[:cutoff]
df_test = df_day[cutoff:]
print(
    f"""Total data: {df_day.shape[0]}
Train data: {df_train.shape[0]}
Test data: {df_test.shape[0]}"""
       )

Total data: 4903
Train data: 3922
Test data: 981


In [21]:
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
from arch import arch_model
from sklearn.metrics import mean_squared_error

returns = np.log(df_day['Close'] / df_day['Close'].shift(1)).dropna()
cutoff = int(len(returns) * 0.8)
train, test = returns[:cutoff], returns[cutoff:]

best_aic = float('inf')
best_bic = float('inf')
best_order_aic = (1, 1)
best_order_bic = (1, 1)
best_model_aic = None
best_model_bic = None
results = []

for p in range(1, 6):
    for q in range(1, 6):
        try:
            model = arch_model(train, vol='GARCH', p=p, q=q)
            res = model.fit(disp='off')
            results.append({'p': p, 'q': q, 'aic': res.aic, 'bic': res.bic})
            if res.aic < best_aic:
                best_aic = res.aic
                best_order_aic = (p, q)
                best_model_aic = res
            if res.bic < best_bic:
                best_bic = res.bic
                best_order_bic = (p, q)
                best_model_bic = res
        except Exception as e:
            print(f'Lỗi với p={p}, q={q}: {e}')

print(f"Best (p, q) theo AIC: {best_order_aic}, AIC: {best_aic}")
print(f"Best (p, q) theo BIC: {best_order_bic}, BIC: {best_bic}")
print("Tóm tắt mô hình tốt nhất theo AIC:")
print(best_model_aic.summary())

d:\python3_11\Lib\site-packages\arch\univariate\base.py:309: DataScaleWarning:

y is poorly scaled, which may affect convergence of the optimizer when
estimating the model parameters. The scale of y is 0.001464. Parameter
estimation work better when this value is between 1 and 1000. The recommended
rescaling is 10 * y.

This warning can be disabled by either rescaling y before initializing the
model or by setting rescale=False.


d:\python3_11\Lib\site-packages\arch\univariate\base.py:309: DataScaleWarning:

y is poorly scaled, which may affect convergence of the optimizer when
estimating the model parameters. The scale of y is 0.001464. Parameter
estimation work better when this value is between 1 and 1000. The recommended
rescaling is 10 * y.

This warning can be disabled by either rescaling y before initializing the
model or by setting rescale=False.


d:\python3_11\Lib\site-packages\arch\univariate\base.py:309: DataScaleWarning:

y is poorly scaled, which may affect convergence of 

Best (p, q) theo AIC: (2, 5), AIC: -16580.645272265327
Best (p, q) theo BIC: (2, 5), BIC: -16524.178354244697
Tóm tắt mô hình tốt nhất theo AIC:
                     Constant Mean - GARCH Model Results                      
Dep. Variable:                  Close   R-squared:                       0.000
Mean Model:             Constant Mean   Adj. R-squared:                  0.000
Vol Model:                      GARCH   Log-Likelihood:                8299.32
Distribution:                  Normal   AIC:                          -16580.6
Method:            Maximum Likelihood   BIC:                          -16524.2
                                        No. Observations:                 3921
Date:                Tue, Jun 03 2025   Df Residuals:                     3920
Time:                        17:56:17   Df Model:                            1
                                 Mean Model                                 
                 coef    std err          t      P>|t|      95.0% C

d:\python3_11\Lib\site-packages\arch\univariate\base.py:309: DataScaleWarning:

y is poorly scaled, which may affect convergence of the optimizer when
estimating the model parameters. The scale of y is 0.001464. Parameter
estimation work better when this value is between 1 and 1000. The recommended
rescaling is 10 * y.

This warning can be disabled by either rescaling y before initializing the
model or by setting rescale=False.


d:\python3_11\Lib\site-packages\arch\univariate\base.py:309: DataScaleWarning:

y is poorly scaled, which may affect convergence of the optimizer when
estimating the model parameters. The scale of y is 0.001464. Parameter
estimation work better when this value is between 1 and 1000. The recommended
rescaling is 10 * y.

This warning can be disabled by either rescaling y before initializing the
model or by setting rescale=False.


d:\python3_11\Lib\site-packages\arch\univariate\base.py:309: DataScaleWarning:

y is poorly scaled, which may affect convergence of 

In [33]:
window_size = len(train)
rolling_forecast = []
rolling_true = []

for i in range(len(test)):
    rolling_train = np.concatenate([train, test[:i]])
    model = arch_model(rolling_train, vol='GARCH', p=best_order_aic[0], q=best_order_aic[1])
    res = model.fit(disp='off')
    forecast = res.forecast(horizon=1, start=len(rolling_train)-1)
    rolling_forecast.append(forecast.variance.iloc[-1, 0])
    rolling_true.append(test.values[i] ** 2)

from sklearn.metrics import mean_squared_error
mse_rolling = mean_squared_error(rolling_true, rolling_forecast)
rmse_rolling = np.sqrt(mse_rolling)
mae_rolling = np.mean(np.abs(np.array(rolling_true) - np.array(rolling_forecast)))
print(f"Rolling GARCH MSE: {mse_rolling:.6f}")
print(f"Rolling GARCH RMSE: {rmse_rolling:.6f}")
print(f"Rolling GARCH MAE: {mae_rolling:.6f}")

d:\python3_11\Lib\site-packages\arch\univariate\base.py:309: DataScaleWarning:

y is poorly scaled, which may affect convergence of the optimizer when
estimating the model parameters. The scale of y is 0.001464. Parameter
estimation work better when this value is between 1 and 1000. The recommended
rescaling is 10 * y.

This warning can be disabled by either rescaling y before initializing the
model or by setting rescale=False.


d:\python3_11\Lib\site-packages\arch\univariate\base.py:309: DataScaleWarning:

y is poorly scaled, which may affect convergence of the optimizer when
estimating the model parameters. The scale of y is 0.001464. Parameter
estimation work better when this value is between 1 and 1000. The recommended
rescaling is 10 * y.

This warning can be disabled by either rescaling y before initializing the
model or by setting rescale=False.


d:\python3_11\Lib\site-packages\arch\univariate\base.py:309: DataScaleWarning:

y is poorly scaled, which may affect convergence of 

Rolling GARCH MSE: 0.000001
Rolling GARCH RMSE: 0.001161
Rolling GARCH MAE: 0.000591


d:\python3_11\Lib\site-packages\arch\univariate\base.py:309: DataScaleWarning:

y is poorly scaled, which may affect convergence of the optimizer when
estimating the model parameters. The scale of y is 0.001258. Parameter
estimation work better when this value is between 1 and 1000. The recommended
rescaling is 10 * y.

This warning can be disabled by either rescaling y before initializing the
model or by setting rescale=False.


d:\python3_11\Lib\site-packages\arch\univariate\base.py:309: DataScaleWarning:

y is poorly scaled, which may affect convergence of the optimizer when
estimating the model parameters. The scale of y is 0.001258. Parameter
estimation work better when this value is between 1 and 1000. The recommended
rescaling is 10 * y.

This warning can be disabled by either rescaling y before initializing the
model or by setting rescale=False.




In [ ]:
# Rolling window dự báo returns và cập nhật giá dự báo từng ngày
rolling_returns_forecast = []
rolling_price_forecast = []
last_price = df_day['Close'].iloc[cutoff-1]

for i in range(len(test)):
    rolling_train = np.concatenate([train, test[:i]])
    model = arch_model(rolling_train, vol='GARCH', p=best_order_aic[0], q=best_order_aic[1])
    res = model.fit(disp='off')
    # Dự báo returns ngày tiếp theo bằng giá trị trung bình (mean) của mô hình
    mean_forecast = res.params['mu'] if 'mu' in res.params else 0
    rolling_returns_forecast.append(mean_forecast)
    # Cập nhật giá dự báo
    if i == 0:
        rolling_price_forecast.append(last_price * np.exp(mean_forecast))
    else:
        rolling_price_forecast.append(rolling_price_forecast[-1] * np.exp(mean_forecast))

# Giá thực tế trên tập test
real_price = df_day['Close'].iloc[cutoff:cutoff+len(test)]

# Đánh giá dự báo giá đóng cửa rolling window
from sklearn.metrics import mean_squared_error, mean_absolute_error
mse_price = mean_squared_error(real_price, rolling_price_forecast)
rmse_price = np.sqrt(mse_price)
mae_price = mean_absolute_error(real_price, rolling_price_forecast)
print(f"Rolling GARCH dự báo giá - MSE: {mse_price:.2f}")
print(f"Rolling GARCH dự báo giá - RMSE: {rmse_price:.2f}")
print(f"Rolling GARCH dự báo giá - MAE: {mae_price:.2f}")